In [2]:
from astropy.io import fits

In [3]:
hdus = fits.open("../tests/data/WFPC2u57.fits")

In [4]:
bytes = open("../tests/data/WFPC2u57.fits", "rb").read()

In [6]:
def get_first_header_size(bytes):
    # Header may be multiple of 2880 bytes, get the size of the header until the block where the END card is present
    header_size = 0
    while True:
        header_size += 2880
        if b"END     " in bytes[header_size:header_size + 2880]:
            header_size += 2880
            break
        
    return header_size


In [7]:
def get_data_size_in_bytes(header):
    ## It may have multiple NAXISn keywords, so we need to get the product of all of them
    size_in_bytes = 1
    for i in range(1, header["NAXIS"] + 1):
        size_in_bytes *= header["NAXIS" + str(i)]
    
    size_in_bytes *= abs(header["BITPIX"] / 8)
    ## Make it multiple of 2880 bytes
    if size_in_bytes % 2880 != 0:
        size_in_bytes += 2880 - (size_in_bytes % 2880)
    
    return int(size_in_bytes)
    

In [8]:
second_hdus_start = get_first_header_size(bytes) + get_data_size_in_bytes(hdus[0].header)
second_hdu_bytes = bytes[second_hdus_start:]

In [9]:
get_first_header_size(second_hdu_bytes) + get_data_size_in_bytes(hdus[1].header)

34560

In [11]:
data_second_hdu = second_hdu_bytes[get_first_header_size(second_hdu_bytes):]

In [13]:
hdus[1].header

XTENSION= 'TABLE   '           / Ascii table extension                          
BITPIX  =                    8 / 8-bits per 'pixels'                            
NAXIS   =                    2 / Simple 2-D matrix                              
NAXIS1  =                  796 / Number of characters per row                   
NAXIS2  =                    4 / The number of rows                             
PCOUNT  =                    0 / No 'random' parameters                         
GCOUNT  =                    1 / Only one group                                 
TFIELDS =                   49 / Number of fields per row                       
EXTNAME = 'u5780205r_cvt.c0h.tab' / Name of table                               
                                                                                
TTYPE1  = 'CRVAL1  '           /                                                
TBCOL1  =                    1 /                                                
TFORM1  = 'D25.17  '        

In [93]:
class Column:
    number: int
    ttype: str
    tform: bytes
    
    def __init__(self, number, ttype, tform):
        self.number = number
        self.ttype = ttype
        self.tform = tform
        self.values = []
    
    def print(self):
        print(f"Column {self.number}: {self.ttype} {self.tform} {self.values}")

In [116]:
header = hdus[1].header
columns = []
for i in range(1, header["TFIELDS"] + 1):
    columns.append(Column(i, header["TTYPE" + str(i)], header["TFORM" + str(i)]))

In [117]:
## if tform is E15.2 I want to get the 15
def get_tform_type_size(tform):
    if len(tform[:1]) == 1:
        size = None
    else:
        size = int(tform[1:].split(".")[0])
        
    return tform[:1], size

In [123]:
pos = 0

for line in range(1, header["NAXIS2"] + 1):
    for column in columns:
        typ, size = get_tform_type_size(column.tform)
        
        if typ == "D":
            column.values.append(float(data_second_hdu[pos:pos + size]))
        elif typ == "E":
            column.values.append(float(data_second_hdu[pos:pos + size]))
        elif typ == "I":
            column.values.append(int(data_second_hdu[pos:pos + size]))
        elif typ == "A":
            column.values.append(data_second_hdu[pos:pos + size].decode("ascii"))
        else:
            column.values.append(data_second_hdu[pos:pos + size])
        
        pos += size + 1

In [124]:
columns[1].print()

Column 2: CRVAL2 D25.17 [39.39633673411, 39.41214313815, 39.410944281389995, 39.38946044168, 39.39633673411, 39.41214313815, 39.410944281389995, 39.38946044168]


In [105]:
from astropy.table import Table
t = Table.read("../tests/data/WFPC2u57.fits", format="fits")
t

CRVAL1,CRVAL2,CRPIX1,CRPIX2,CD1_1,CD1_2,CD2_1,CD2_2,DATAMIN,DATAMAX,MIR_REVR,ORIENTAT,FILLCNT,ERRCNT,FPKTTIME,LPKTTIME,CTYPE1,CTYPE2,DETECTOR,DEZERO,BIASEVEN,BIASODD,GOODMIN,GOODMAX,DATAMEAN,GPIXELS,SOFTERRS,CALIBDEF,STATICD,ATODSAT,DATALOST,BADPIXEL,OVERLAP,PHOTMODE,PHOTFLAM,PHOTZPT,PHOTPLAM,PHOTBW,MEDIAN,MEDSHADO,HISTWIDE,SKEWNESS,MEANC10,MEANC25,MEANC50,MEANC100,MEANC200,MEANC300,BACKGRND
,,,,,,,,,,LOGICAL-,,,,,,CHARACTER*8,CHARACTER*8,,,,,,,,,,,,,,,,CHARACTER*48,,,,,,,,,,,,,,,
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes1,float64,int64,int64,float64,float64,bytes8,bytes8,int64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,int64,int64,int64,bytes48,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
182.6311886308,39.39633673411,420,424.5,-1.06704e-06,-1.25958e-05,-1.26016e-05,1.06655e-06,-73.19537,3777.701,T,-85.16,0,0,51229.79857399999,51229.79874200001,RA---TAN,DEC--TAN,1,316.6452,316.6715,316.6189,-5.064006,2552.17,0.4182382,632387,0,1466,0,16,0,0,0,"WFPC2,1,A2D7,LRF#4877.0,,CAL",3.44746e-16,-21.1,4884.258,20.20996,-0.175651,-0.121681,1.033711,-1.983727,0.12958,0.3129676,0.4577668,0.3916293,0.3115222,0.3295493,-0.3676353
182.6255233634,39.41214313815,423.5,414,-2.75605e-05,2.08221e-06,2.08021e-06,2.75871e-05,-19.95751,3740.623,T,4.31638,0,0,51229.79874200001,51229.79891000002,RA---TAN,DEC--TAN,2,354.0573,354.0761,354.0386,-3.379329,3720.573,0.6385469,631340,0,2072,0,25,0,0,0,"WFPC2,2,A2D7,LRF#4877.0,,CAL",3.382652e-16,-21.1,4884.261,20.20992,-0.1896807,-0.1899146,1.053666,-1.947547,0.2564496,0.1057321,0.9952926,0.8409855,0.6534808,0.5836062,0.2101429
182.6523792305,39.41094428138999,436,424.5,2.23913e-06,2.75426e-05,2.75684e-05,-2.23704e-06,-58.53249,3282.136,T,94.6434,0,0,51229.79891000002,51229.799078,RA---TAN,DEC--TAN,3,306.6676,306.6541,306.681,-3.713645,3282.136,0.5217843,633259,0,1551,0,7,0,0,0,"WFPC2,3,A2D7,LRF#4877.0,,CAL",3.372288e-16,-21.1,4884.255,20.21006,0.01109185,0.06184989,1.003236,28.21703,1.669304,0.3472891,0.327215,0.6145074,0.7027547,0.5681439,0.4761558
182.650022355,39.38946044168,423,421,2.75358e-05,-2.50931e-06,-2.50689e-06,-2.75624e-05,-72.41166,3420.24,T,-174.798,0,0,51229.799079,51229.79924599999,RA---TAN,DEC--TAN,4,311.7072,311.7648,311.6496,-3.289304,3420.24,0.7084697,628289,0,1816,0,57,0,0,0,"WFPC2,4,A2D7,LRF#4877.0,,CAL",3.46466e-16,-21.1,4884.256,20.20999,-0.06813275,-0.03253493,0.9846212,-4.591252,0.1359666,0.04015671,0.1129934,1.897491,0.9687142,1.05377,0.3466465
